# Pokémon TCG Synergy Engine
---

In [1]:
# import libraries for data wrangling
import ast
import re
import pandas as pd
import numpy as np
import src

## Pokémon TCG Data Wrangling
---

In [2]:
data = src.get_card_data()
df_pokemon = src.prep_pokemon_data(data)
df_trainers = src.prep_trainer_df(data)
df_energy = src.prep_energy_df(data)

In [3]:
df_pokemon

,id,set_name,set_number,supertype,name,stage,is_ex,is_tera,primary_type,evolves_from,...,attack_damage_modifier,attack_cost,cards_needed_for_attack,attack_energy_cost,is_coin_flip,damage_per_energy,retreat_cost,prize_card_value,setup_time,is_immune_to_bench_damage
0,sv1-1,sv1,1,Pokémon,Pineco,Basic,0,0,Grass,NaN,...,1.0,"['Colorless', 'Colorless']",2,2,False,5.00,2.0,1,0,0
1,sv1-2,sv1,2,Pokémon,Heracross,Basic,0,0,Grass,NaN,...,1.0,"['Grass', 'Colorless']",2,2,False,5.00,2.0,1,0,0
2,sv1-2,sv1,2,Pokémon,Heracross,Basic,0,0,Grass,NaN,...,9.0,"['Grass', 'Grass', 'Colorless']",3,3,False,30.00,2.0,1,0,0
3,sv1-3,sv1,3,Pokémon,Shroomish,Basic,0,0,Grass,NaN,...,1.0,['Grass'],1,1,False,10.00,1.0,1,0,0
4,sv1-4,sv1,4,Pokémon,Breloom,Stage 1,0,0,Grass,Shroomish,...,6.0,['Grass'],2,1,False,60.00,1.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2881,svp-157,svp,157,Pokémon,Zapdos,Basic,0,0,Lightning,NaN,...,2.0,"['Lightning', 'Colorless']",2,2,False,10.00,1.0,1,0,0
2882,svp-157,svp,157,Pokémon,Zapdos,Basic,0,0,Lightning,NaN,...,8.0,"['Lightning', 'Colorless', 'Colorless']",3,3,False,26.67,1.0,1,0,0
2883,svp-158,svp,158,Pokémon,Pachirisu,Basic,0,0,Lightning,NaN,...,NaN,['Colorless'],1,1,True,NaN,1.0,1,0,0
2884,svp-158,svp,158,Pokémon,Pachirisu,Basic,0,0,Lightning,NaN,...,3.0,"['Lightning', 'Colorless']",2,2,False,15.00,1.0,1,0,0


In [4]:
import re

def tag_trainer_card(rules):
    if not isinstance(rules, list):
        return []

    tags = set()
    joined_rules = " ".join(rules).lower() if rules else ""

    if "draw" in joined_rules:
        tags.add("draw")
    if "shuffle" in joined_rules:
        tags.add("shuffle")
    if "discard" in joined_rules:
        tags.add("discard")
    if re.search(r"switch.*your opponent", joined_rules):
        tags.add("gust")
    if "heal" in joined_rules:
        tags.add("heal")
    if "supporter" in joined_rules:
        tags.add("supporter")
    if "item" in joined_rules:
        tags.add("item")
    if "stadium" in joined_rules:
        tags.add("stadium")
    if "tool" in joined_rules:
        tags.add("tool")
    if "ace spec" in joined_rules:
        tags.add('ace_spec')

    if "search your deck" in joined_rules:
        tags.add("search")  # general fallback
        if "basic pokémon" in joined_rules:
            tags.add("search_basic")
        if "evolution pokémon" in joined_rules or "evolution card" in joined_rules:
            tags.add("search_evolution")
        if "pokémon" in joined_rules and not ("basic" in joined_rules or "evolution" in joined_rules):
            tags.add("search_pokemon")
        if "energy card" in joined_rules:
            tags.add("search_energy")
        if "trainer card" in joined_rules:
            tags.add("search_trainer")
        if "supporter card" in joined_rules:
            tags.add("search_supporter")
        if "tool card" in joined_rules:
            tags.add("search_tool")
        if "stadium card" in joined_rules:
            tags.add("search_stadium")

    if "evolve" in joined_rules or "put onto 1 of your" in joined_rules:
        tags.add("evolve")

    return list(tags)

In [5]:
df_trainers['trainer_tags'] = df_trainers['rules'].apply(tag_trainer_card)

In [6]:
df_trainers

,id,supertype,subtype,name,is_ace_spec,is_future,is_ancient,rules,abilities,hp,set_number,set_name,trainer_tags
0,sv1-198,Trainer,Supporter,Youngster,0,0,0,"[Shuffle your hand into your deck. Then, draw ...",NaN,NaN,198,sv1,"[shuffle, supporter, draw]"
1,sv6pt5-64,Trainer,Supporter,Xerosic's Machinations,0,0,0,[Your opponent discards cards from their hand ...,NaN,NaN,64,sv6pt5,"[discard, supporter]"
2,sv1-197,Trainer,Pokémon Tool,Vitality Band,0,0,0,[The attacks of the Pokémon this card is attac...,NaN,NaN,197,sv1,[tool]
3,sv3-197,Trainer,Pokémon Tool,Vengeful Punch,0,0,0,[If the Pokémon this card is attached to is Kn...,NaN,NaN,197,sv3,[tool]
4,sv6-165,Trainer,Item,Unfair Stamp,1,0,0,[You can't have more than 1 ACE SPEC card in y...,NaN,NaN,165,sv6,"[ace_spec, item, shuffle, draw]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,sv4-159,Trainer,Pokémon Tool,Ancient Booster Energy Capsule,0,0,1,[The Ancient Pokémon this card is attached to ...,NaN,NaN,159,sv4,[tool]
242,sv8-162,Trainer,Pokémon Tool,Amulet of Hope,1,0,0,[If the Pokémon this card is attached to is Kn...,NaN,NaN,162,sv8,"[search_pokemon, tool, ace_spec, search, shuffle]"
243,sv8pt5-93,Trainer,Supporter,Amarys,0,0,0,"[Draw 4 cards. At the end of this turn, if you...",NaN,NaN,93,sv8pt5,"[discard, supporter, draw]"
244,sv6-142,Trainer,Item,Accompanying Flute,0,0,0,[Reveal the top 5 cards of your opponent's dec...,NaN,NaN,142,sv6,"[item, shuffle]"
